In [1]:
# Step 2. 데이터 읽어오기

import glob
import os, re 
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [2]:
# Step 3. 데이터 정제

# 입력된 문장 변경
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 여기에 정제된 문장을 모은다
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0 : continue
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) > 14 : continue
    corpus.append(preprocessed_sentence)
        
# 정제된 결과 확인
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [3]:
# Step 4. 평가 데이터셋 분리

# 토큰화, 데이터를 tensor로 변환
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer 만든다
    # 우리는 이미 문장을 정제했으니 filters가 필요없음
    # 포함되지 못한 단어는 '<unk>'로
    # maxlen 으로 텍스트 길이 제한
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, filters=' ', oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 3393 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  119 ...    0    0    0]
 [   2  254  193 ...   12    3    0]
 [   2    7   35 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe7843f0f70>


In [4]:
# tokenizer 인덱스 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [5]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   98  299   61   52    9  953 6639    3    0    0    0]
[  50    4   98  299   61   52    9  953 6639    3    0    0    0    0]


In [6]:
# 데이터셋 객체 생성해보자
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만들자!
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [7]:
# Step 4. 평가 데이터셋 분리

from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=20)

In [8]:
print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (120158, 14)
Target Train:  (120158, 14)


Step 5. 인공지능 만들기
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)
그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [9]:
# 모델 생성하기
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

# 모델 학습하기
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
3755/3755 [==============================] - 143s 36ms/step - loss: 2.9729 - val_loss: 2.7221
Epoch 2/10
3755/3755 [==============================] - 142s 38ms/step - loss: 2.5371 - val_loss: 2.5185
Epoch 3/10
3755/3755 [==============================] - 142s 38ms/step - loss: 2.2424 - val_loss: 2.3920
Epoch 4/10
3755/3755 [==============================] - 142s 38ms/step - loss: 1.9758 - val_loss: 2.3167
Epoch 5/10
3755/3755 [==============================] - 142s 38ms/step - loss: 1.7463 - val_loss: 2.2828
Epoch 6/10
3755/3755 [==============================] - 142s 38ms/step - loss: 1.5555 - val_loss: 2.2679
Epoch 7/10
3755/3755 [==============================] - 142s 38ms/step - loss: 1.4029 - val_loss: 2.2753
Epoch 8/10
3755/3755 [==============================] - 142s 38ms/step - loss: 1.2844 - val_loss: 2.2947
Epoch 9/10
3755/3755 [==============================] - 142s 38ms/step - loss: 1.1939 - val_loss: 2.3247
Epoch 10/10
3755/3755 [==============================] 

In [12]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

lyricist = model
generate_text(lyricist, tokenizer, init_sentence="<start> i like", max_len=20)

'<start> i like the way how you re lovin me <end> '